# Everyone should use Pandas! (and sometimes astropy tables)
* This tutorial is meant to get you started with using Pandas and Astropy tables to organize and work with your data sets 

* Storing your data as pandas dataframes makes querying and sorting your data, merging different data sets, doing statistics, and visualiziing your data faster, easier, and cleaner. 
* Pandas also handels messy data and data with missing fields very well 
* Pandas is a python package widely used to build and work with dataframes. Astropy has a class called Tables which has similar functionality but much more limited and less supported. I introduce it breifly here because it has a few astronomy specific features absent in Pandas that are sometimes useful. For most things the two objects can be used interchangably. 

## Main documentation for both Pandas, Astropy tables, and Astroquery
* [Pandas](http://pandas.pydata.org/pandas-docs/stable/)
* [Astropy Tables](http://docs.astropy.org/en/stable/table/)
* [Astroquery](https://astroquery.readthedocs.io/en/latest/)

In [19]:
#import all needed python packages
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from astropy import units as u
from astropy.coordinates import SkyCoord

#comment out the next two lines if you do not have astroquery installed
from astroquery.sdss import SDSS
from astroquery.vizier import Vizier

from astropy.table import Table, Column, join
import pandas as pd

## Types of Tables in Pandas
1. Series - 1D array, can be build out of a dictionary 
2. Dataframes - 2D array 

### Tables are different from numpy arrays:
* Each row in a table is assigned an index. 
* In a dataframe column also has a name 
* A numpy array can also be indexed but you can't change what the indecies are named 
    (always indexed in the same way)
* tables can have columns of all different data types 

In [23]:
np_array = np.array([1,2,3,4,5])

print('Numpy Array')
print(np_array)
print('\n')

#It will assign the index for you if you don't give it one
#Indexing always starts at 0 in python
pd_series = pd.Series([1,2,3,4,5])

print('Pandas Series')
print(pd_series)

### Series in Pandas

In [24]:
#You can define the index for the series 
s1 = pd.Series([1,2,3,4,5],index = ['a','b','c','d','e'])

#building a series from a dictionary 
data = {'cats' : 15., 'dogs' : 35., 'turtles' : 100.}
s2 = pd.Series(data)

#You can assign the index (pandas handels missing data very well)
s3 = pd.Series(data,index=['turtles','cats','birds','dogs'])

s3

### Dataframes in Pandas

In [25]:
#example of how to build a dataframe
#Example: I have an array of data that tells me the number of animals on a given date
dates = pd.date_range('20180620', periods=6)
rand_data = np.random.randint(low=0, high=100, size=(6,4))
df = pd.DataFrame(rand_data, index=dates, columns=['cats','dogs','birds','turtles'])

print(df)
print('\n')

#If you don't care what the index is you don't have to give it one
#If you don't assign and index it just gives them index values from 0 to # rows

df = pd.DataFrame(rand_data, columns=['cats','dogs','birds','turtles'])

print(df)

## Importing data with Pandas 
* Pandas can easily read in files of data in one line 
* Unless you specify the dtypes you want for each column it chooses the best match 
* Each read function in Pandas is very flexible with a lot of arguments to to read in a lot of differnet formats
* Generic delimited text [(documentation)](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html)
    * `read_csv()` 
    * `read_table()` - defaults to tab separated

### Other formats
Pandas has built-in support for [reading from or to other common formats/sources](http://pandas.pydata.org/pandas-docs/stable/io.html):
* Excel -- `read_excel()`
* JSON -- `read_json()`
* SQL -- `read_sql()`
* Stata -- `read_stata()`
* SAS (XPORT or SAS7BDAT) -- `read_sas()`
* etc...

## We are going to work with the HETDEX Pilot Survey emission line catalog
* this is a Vizier catalog that can be found [here](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=J/ApJS/192/5/hetdex)
* The page has an option to save the table as a tab separated file which I did 

In [53]:
#importing data with pandas 

#Because the file has such a large header you need to tell it which row you want it to use for the column names (header)
#Tell it to skip the rows below it that have the units and the line 
#If you don't care about the header information it is sometimes easier to just delete it from the top of the file
#  if you delete the header just make sure to leave the top row of column names
hps_pd = pd.read_table('hps_catalog.txt', delimiter=None, header=62, skiprows=[64,65])
hps_pd

### Astropy supports different file formats (commonly used in astronomy) that Pandas doesn't
* some of the formats are more astronomy specific (ex. fits)
* It does allow you to save tables as deluxe latex tables which is extremely useful!
* It also will read in a row of units for columns (useful when reading in catalogs)
* A [list of the format Astropy supports](http://docs.astropy.org/en/stable/io/unified.html#built-in-readers-writers)

In [30]:
#importing data with astropy tables

hps_at = Table.read('hps_catalog.txt', format='ascii')
hps_at


## Reading in with Pandas vs. Astropy Tables 
* You can see that Tables did not automatically pick up on there being two rows under the column names that are not data
* the Tables read function does not have arguments for formating files like pandas 
* so unless your data fits the formats provided astropy tables can be annoying to work with 
* I usually have to format the file before I try to read it in with astropy tables 
* Pandas provides a lot more flexlibilty 

## However, you can also load it in with Astropy Tables using Astroquery
* Astroquery allows you to load in a catalog from a bunch of different sources 
* [Here](https://astroquery.readthedocs.io/en/latest/#catalog-archive-and-other) is a list of all of the catalogs and archives that now work with astroquery (some are better supported than others)
* [Here](http://astroquery.readthedocs.io/en/latest/gallery.html) is a page with astroquery examples for some of the databases 

In [34]:
#queries with astropy (Vizier)

hps_catalog_list = Vizier.find_catalogs('J/ApJS/192/5/')
hps_catalogs = Vizier.get_catalogs(hps_catalog_list.keys())
hps_at = hps_catalogs[0]

hps_at
#hps_at.show_in_notebook() #This is a tool for viewing astropy tables in a Jupyter notebook
#hps_at.show_in_browser(jsviewer=True) #this is a tool for viewing a searchable astropy table in a web browser

## Interfacing between pandas and astropy tables 
* you can covert from one to the other very easily with the following commands 

In [35]:
# astropy tables to pandas
hps_pd_from_at = hps_at.to_pandas()

# pandas to astropy table
hps_at_from_pd = Table.from_pandas(hps_pd)

# **From now on in this tutorial we will be using Pandas**
* some of this functionality exists in Astropy Tables but not all of it. 

## Exploring your data set 

In [36]:
#hps_pd.head()
hps_pd.tail(10)

,HPS,lambda,FWHM,S/N,Flux,Spat,mag,n_mag,Prob,EW,EWint,Trans,z,Lya,X-ray,RAJ2000,DEJ2000
469,470,5823.58,274,12.6,20.1,5.1,22.19,R,0.98,30.2,42.3,[OII],0.5625,0.01,,12 37 08.24,+62 14 55.0
470,471,5258.97,287,14.6,24.7,5.0,21.01,R,0.94,11.3,21.1,[OII],0.4110,0.00,,12 37 08.76,+62 10 52.1
471,472,5527.22,217,5.6,3.9,3.1,23.29,R,0.47,15.4,23.9,[OII],0.4830,0.03,,12 37 10.81,+62 10 50.2
472,473,5497.45,478,5.9,7.5,3.5,23.16,R,0.96,25.8,46.7,[OII],0.4750,0.02,,12 37 10.93,+62 11 30.5
473,474,3980.21,564,5.6,13.6,4.2,25.14,R,0.23,68.6,102.0,Ly{alpha},2.2741,0.28,,12 37 11.00,+62 11 40.9
474,475,5256.73,427,5.6,11.2,4.0,21.94,R,0.83,12.1,23.0,[OII],0.4104,0.08,,12 37 11.50,+62 11 29.4
475,476,5264.56,440,12.7,19.1,4.0,22.02,R,0.97,22.1,38.0,[OII],0.4125,0.03,,12 37 11.60,+62 10 21.1
476,477,4840.35,303,38.1,70.8,5.1,20.09,R,1.00,12.8,25.4,[OII],0.2987,0.00,,12 37 11.68,+62 15 14.5
477,478,5023.20,334,6.2,7.2,4.1,20.09,R,1.00,1.4,2.4,[NeIII]3869,0.2983,0.00,,12 37 11.69,+62 15 14.4
478,479,5677.75,295,8.3,15.5,5.2,21.67,R,0.97,14.0,24.6,[OII],0.5234,0.03,,12 37 13.52,+62 15 11.6


In [40]:
hps_pd.columns

Index([u'HPS', u'lambda', u'FWHM', u'S/N', u'Flux', u'Spat', u'mag', u'n_mag',
       u'Prob', u'EW', u'EWint', u'Trans', u'z', u'Lya', u'X-ray', u'RAJ2000',
       u'DEJ2000'],
      dtype='object')

In [41]:
hps_pd.columns.values

array(['HPS', 'lambda', 'FWHM', 'S/N', 'Flux', 'Spat', 'mag', 'n_mag',
       'Prob', 'EW', 'EWint', 'Trans', 'z', 'Lya', 'X-ray', 'RAJ2000',
       'DEJ2000'], dtype=object)

In [38]:
hps_pd.dtypes

HPS          int64
lambda     float64
FWHM         int64
S/N        float64
Flux       float64
Spat       float64
mag         object
n_mag       object
Prob       float64
EW         float64
EWint      float64
Trans       object
z          float64
Lya        float64
X-ray       object
RAJ2000     object
DEJ2000     object
dtype: object

In [39]:
hps_pd.describe()

,HPS,lambda,FWHM,S/N,Flux,Spat,Prob,EW,EWint,z,Lya
count,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000,479.000000
mean,240.000000,4986.923048,352.816284,13.220251,49.998956,5.017537,0.868977,59.308559,114.145929,0.860039,0.223027
std,138.419652,531.031005,266.795684,12.699558,124.107192,1.205802,0.220255,224.945405,396.408103,1.062051,0.385620
min,1.000000,3649.390000,74.000000,5.000000,3.200000,1.300000,0.000000,0.100000,0.200000,0.058500,0.000000
25%,120.500000,4591.200000,219.500000,6.600000,14.450000,4.300000,0.870000,8.100000,15.900000,0.197650,0.000000
50%,240.000000,5079.540000,305.000000,8.700000,24.100000,4.900000,0.980000,17.300000,31.400000,0.378900,0.010000
75%,359.500000,5434.565000,390.500000,14.900000,41.600000,5.700000,1.000000,40.500000,62.450000,0.549150,0.120000
max,479.000000,5823.580000,2323.000000,104.500000,1903.500000,11.000000,1.000000,3774.900000,5461.000000,3.743800,1.000000


## Indexing Pandas dataframes
* You can either index by the row and column number as you with in numpy 
* Or more commonly you use the columns names and the row index value
* Pandas defults to using label indexing
* For clarity pandas makes you specify if you using integer or label indexing
    * this is neccessary because the columns names are allowed to be numbers 
    * without specifying pandas would not know if you mean column number 1 or the column named 1 

In [15]:
#label indexing

#hps_pd['EW']
#hps_pd.EW
hps_pd.loc[:,'EW']

0       51.9
1       42.0
2       58.8
3       10.4
4       54.9
5       56.7
6       31.2
7       51.2
8       51.0
9       24.0
10      40.8
11       5.1
12      62.9
13      39.3
14      10.7
15      17.0
16      36.4
17      14.4
18      24.3
19      22.7
20      29.5
21     103.4
22      27.1
23      14.6
24     158.9
25      28.8
26      29.4
27      12.3
28      21.9
29       8.6
       ...  
449     22.2
450      4.1
451      5.2
452      0.7
453     33.3
454      1.6
455      3.4
456      2.8
457      7.0
458     35.2
459     18.9
460     54.2
461    195.4
462     32.0
463      6.7
464     30.0
465     31.0
466    270.7
467     39.6
468     17.1
469     30.2
470     11.3
471     15.4
472     25.8
473     68.6
474     12.1
475     22.1
476     12.8
477      1.4
478     14.0
Name: EW, Length: 479, dtype: float64

In [46]:
#You can ask for certian rows in that column
hps_pd['EW'][2:5] 

2    58.8
3    10.4
4    54.9
Name: EW, dtype: float64

In [16]:
#can also get more than one column
hps_pd.loc[:25,['Trans','Lya','S/N']]

,Trans,Lya,S/N
0,[OII],0.07,8.1
1,[OII],0.04,5.6
2,Ly{alpha},1.00,7.5
3,CIII]1909,0.02,6.3
4,CIV1549,0.00,33.1
5,Ly{alpha},1.00,14.8
6,[OII],0.02,19.5
7,[OII],0.01,6.7
8,[OII],0.02,12.1
9,[OII],0.01,15.9


In [48]:
#You can also use the query function
#hps_pd.query('(z < 0.5) & (EW > 50.)')

#can also write it this way 
hps_pd.loc[(hps_pd.z < 0.5) & (hps_pd.EW > 50)]


,HPS,lambda,FWHM,S/N,Flux,Spat,mag,n_mag,Prob,EW,EWint,Trans,z,Lya,X-ray,RAJ2000,DEJ2000
0,1,5219.16,229,8.1,17.4,4.7,23.05,R,0.93,51.9,62.9,[OII],0.4004,0.07,,02 21 11.16,-04 31 25.0
8,9,5464.33,78,12.1,14.1,3.6,23.21,R,0.98,51.0,49.0,[OII],0.4661,0.02,,02 21 17.25,-04 30 10.4
32,33,5460.20,343,9.4,25.4,5.0,22.59,R,0.54,51.6,69.2,[OII],0.4650,0.01,,02 21 26.83,-04 30 05.8
148,149,4995.67,449,6.3,24.3,7.3,22.85,R,0.88,57.1,106.7,[OII],0.3404,0.12,,10 00 07.14,+02 17 06.3
233,234,5465.91,78,14.1,24.4,4.5,23.37,R,0.63,136.8,133.8,[OIII]5007,0.0917,0.37,,10 00 21.49,+02 13 51.5
288,289,5235.12,428,6.4,17.1,6.5,23.37,R,0.63,68.4,146.0,[OII],0.4046,0.37,,10 00 35.24,+02 18 07.3
316,317,5510.02,149,9.2,17.0,4.6,23.08,R,0.98,54.8,62.7,[OII],0.4784,0.02,,10 00 42.92,+02 18 46.0
336,337,5355.81,205,7.6,12.3,4.9,23.53,R,0.89,58.2,93.4,[OII],0.4370,0.11,,12 36 16.55,+62 13 11.2


In [18]:
#You can also index by location like you do in numpy using iloc

#If you give it a single value it returns a series of that row number 
hps_pd.iloc[0]

HPS                       1
lambda              5219.16
FWHM                    229
S/N                     8.1
Flux                   17.4
Spat                    4.7
mag                   23.05
n_mag                     R
Prob                   0.93
EW                     51.9
EWint                  62.9
Trans           [OII]      
z                    0.4004
Lya                    0.07
X-ray                      
RAJ2000         02 21 11.16
DEJ2000         -04 31 25.0
Name: 0, dtype: object

In [49]:
# You can also give it a specific section of the table you want 
#format is [row_start:row_end, col_start:col_end]
hps_pd.iloc[0:10, 2:6]

,FWHM,S/N,Flux,Spat
0,229,8.1,17.4,4.7
1,307,5.6,12.2,4.3
2,422,7.5,19.9,4.4
3,1285,6.3,42.6,5.1
4,1841,33.1,342.1,4.8
5,399,14.8,32.7,4.6
6,293,19.5,49.4,4.7
7,118,6.7,19.1,6.6
8,78,12.1,14.1,3.6
9,357,15.9,38.9,4.6


In [50]:
#It will also take a list of column numbers 
hps_pd.iloc[0:10, [1,4,6,9]]

,lambda,Flux,mag,EW
0,5219.16,17.4,23.05,51.9
1,5448.72,12.2,23.17,42.0
2,4973.93,19.9,24.31,58.8
3,5261.37,42.6,21.05,10.4
4,4270.67,342.1,21.05,54.9
5,4591.58,32.7,23.82,56.7
6,5161.72,49.4,21.38,31.2
7,5820.13,19.1,22.82,51.2
8,5464.33,14.1,23.21,51.0
9,4808.33,38.9,21.43,24.0


## Useful Functions
* Pandas has a lot of useful built in functions 
* Pandas apply function allows you to operate a function on all values in a column/table

In [55]:
hps_pd['Trans'].values

In [52]:
#You may notice when it read in the Trans column it added extra spaces making it difficult to query
# You can use the apply function to strip each value in the Trans column 
# Anonymous (lambda) funcitons in python are a convienent way to write a one line function to operate over a variable
# the strip() function strips a string of its whitespace
hps_pd['Trans'] = hps_pd['Trans'].apply(lambda x: x.strip())

#can ues the values function to return an array of just a columns values 
hps_pd['Trans'].values

array(['[OII]', '[OII]', 'Ly{alpha}', 'CIII]1909', 'CIV1549', 'Ly{alpha}',
       '[OII]', '[OII]', '[OII]', '[OII]', 'Ly{alpha}', '[OII]',
       'Ly{alpha}', '[OII]', '[OII]', '[OII]', 'Ly{alpha}', '[NeIII]3869',
       '[OII]', '[OII]', '[OII]', 'Ly{alpha}', '[OII]', '[OII]',
       'Ly{alpha}', '[OII]', '[OIII]5007', 'H{beta}', '[OII]',
       '[OIII]4959', '[OII]', '[OII]', '[OII]', 'Ly{alpha}', '[OII]',
       'H{gamma}', 'H{beta}', '[OIII]5007', '[OIII]4959', '[OII]',
       '[OII]', '[OII]', '[OIII]5007', '[OII]', '[OII]', '[OII]', '[OII]',
       '[OII]', '[OII]', '[OII]', 'Ly{alpha}', '[OII]', 'Ly{alpha}',
       '[OII]', '[OII]', '[OII]', '[OII]', '[OII]', '[OII]', '[OII]',
       '[OII]', 'Ly{alpha}', '[OII]', '[OII]', '[OII]', 'H{beta}',
       '[OII]', '[OIII]5007', '[OII]', '[OII]', 'H{delta}', '[NeV]3426',
       '[OII]', '[NeIII]3869', '[OIII]4959', 'H{beta}', '[OIII]5007',
       'H{gamma}', '[OII]', '[OII]', '[OII]', 'Ly{alpha}', '[OII]',
       'Ly{alpha}', 'MgII279

In [ ]:
#find the unique types of emission line types 
emission_lines = hps_pd['Trans'].unique()
emission_lines

In [ ]:
#get an table of HPS ids, fluxes, and S/N for Ly{alpha} lines where the probablilty of it being Lya is greater than 50%
hps_pd.query('(Trans == "Ly{alpha}") & (Lya > 0.5)').loc[:,['HPS','Flux','S/N']]

In [ ]:
# you want the lya line with the largest flux 
max_lya_flux = hps_pd.query('(Trans == "Ly{alpha}") & (Lya > 0.5)')['Flux'].max()
hps_pd.loc[hps_pd['Flux'] == max_lya_flux]
#hps_pd.iloc[max_lya_flux]

In [ ]:
#want to know how many of each line
hps_pd['Trans'].value_counts()

In [ ]:
#count the number of LyA lines with a probablility greater than 50%
hps_pd.query('(Trans == "Ly{alpha}") & (Lya > 0.5)')['HPS'].count()

In [ ]:
#sort the LyA lines by the flux values in descending order 
LyA_sorted_by_flux = hps_pd.query('Trans == "Ly{alpha}"').sort_values('Flux', ascending=False)
LyA_sorted_by_flux

In [ ]:
#build function to give to apply for a column
def AA_to_nm(wl):
    wl = wl/10
    return wl

hps_pd['lambda'].apply(AA_to_nm)

In [ ]:
#Here is a function that builds all of the RAj2000 and DEJ2000 values into actual astropy coordinates
#It stores the coordinates in a new column in the dataframe called coord

def build_astropy_coords(row):
    ra  = row['RAJ2000'].split()
    dec = row['DEJ2000'].split()
    c = SkyCoord(ra[0]+'h'+ra[1]+'m'+ra[2]+'s', dec[0]+'d'+dec[1]+'m'+dec[2]+'s', frame='icrs')
    return c

#You give apply the function and the argument axis=1 to it feeds the function the rows of the dataframe
#It stores these new values in a new column called coord
hps_pd['coord'] = hps_pd.apply(build_astropy_coords, axis=1)

#prints the new ra and dec of the stored coordinate for the first row
#print(hps_pd['coord'][0].ra, hps_pd['coord'][0].dec)

hps_pd.head()

In [ ]:
#importing data with pandas 

#Because the file has such a large header you need to tell it which row you want it to use for the column names (header)
#Tell it to skip the rows below it that have the units and the line 
hps_pd = pd.read_table('hps_catalog.txt', delimiter=None, header=62, skiprows=[64,65])
hps_pd

In [6]:
#want to find all of the rows that have an x-ray detection 

#first want to replace all of the rows with nothing but spaces in the X-ray column with nan values 
hps_pd['X-ray'] = hps_pd['X-ray'].replace('                ', np.nan, regex=True)

#use the drapna function to remove the rows without a detection. 
#the .index function returns the indecies of the columns with an x-ray detection
#Then use the .loc function to find the rows with that index 
possible_AGN = hps_pd.loc[hps_pd['X-ray'].dropna().index]
possible_AGN

,HPS,lambda,FWHM,S/N,Flux,Spat,mag,n_mag,Prob,EW,EWint,Trans,z,Lya,X-ray,RAJ2000,DEJ2000
3,4,5261.37,1285,6.3,42.6,5.1,21.05,R,0.98,10.4,7.1,CIII]1909,1.7561,0.02,J0221151-043156,02 21 14.86,-04 31 56.6
4,5,4270.67,1841,33.1,342.1,4.8,21.05,R,1.00,54.9,55.1,CIV1549,1.7570,0.00,J0221151-043156,02 21 15.14,-04 31 54.0
51,52,5310.40,374,19.4,110.7,8.7,17.81,i,1.00,2.7,8.5,[OII],0.4248,0.00,J0306351+000004,03 06 35.18,+00 00 03.2
67,68,5564.20,410,5.5,67.0,8.6,20.47,i,0.63,26.3,43.5,[OIII]5007,0.1113,0.00,J0306393-000022,03 06 38.94,-00 00 22.0
70,71,4536.81,1840,6.0,110.7,1.5,16.09,i,1.00,0.5,0.9,H{delta},0.1060,0.00,J0306395+000343,03 06 39.51,+00 03 44.7
71,72,3791.53,582,8.5,200.9,4.2,16.09,i,1.00,0.7,1.4,[NeV]3426,0.1067,0.00,J0306395+000343,03 06 39.53,+00 03 45.3
72,73,4126.75,356,36.4,437.0,4.8,16.09,i,1.00,1.7,3.3,[OII],0.1073,0.00,J0306395+000343,03 06 39.53,+00 03 45.0
73,74,4282.36,885,26.4,449.6,4.5,16.09,i,1.00,1.9,3.5,[NeIII]3869,0.1068,0.00,J0306395+000343,03 06 39.54,+00 03 44.6
74,75,5490.35,504,66.0,583.2,3.9,16.09,i,1.00,4.0,6.2,[OIII]4959,0.1071,0.00,J0306395+000343,03 06 39.54,+00 03 44.2
75,76,5381.88,1187,83.5,1264.5,3.4,16.09,i,1.00,8.3,13.3,H{beta},0.1072,0.00,J0306395+000343,03 06 39.54,+00 03 44.2


In [7]:
#the dropna function had many arguments that make it very powerful. 

hps_drop_allNaN  = hps_pd.dropna()     #drop all rows that have any NaN values
hps_drop_onlyNaN = hps_pd.dropna(how='all')     #drop only if ALL columns are NaN
hps_drop_someNaN = hps_pd.dropna(thresh=3)   #Will only keep rows with 3 or more **not** NaN values

#this is a better way of writing the line defining possible_AGN in the previous cell
possible_AGN = hps_pd.dropna(subset=['X-ray'])   #Drop only if NaN in specific column 
possible_AGN

,HPS,lambda,FWHM,S/N,Flux,Spat,mag,n_mag,Prob,EW,EWint,Trans,z,Lya,X-ray,RAJ2000,DEJ2000
3,4,5261.37,1285,6.3,42.6,5.1,21.05,R,0.98,10.4,7.1,CIII]1909,1.7561,0.02,J0221151-043156,02 21 14.86,-04 31 56.6
4,5,4270.67,1841,33.1,342.1,4.8,21.05,R,1.00,54.9,55.1,CIV1549,1.7570,0.00,J0221151-043156,02 21 15.14,-04 31 54.0
51,52,5310.40,374,19.4,110.7,8.7,17.81,i,1.00,2.7,8.5,[OII],0.4248,0.00,J0306351+000004,03 06 35.18,+00 00 03.2
67,68,5564.20,410,5.5,67.0,8.6,20.47,i,0.63,26.3,43.5,[OIII]5007,0.1113,0.00,J0306393-000022,03 06 38.94,-00 00 22.0
70,71,4536.81,1840,6.0,110.7,1.5,16.09,i,1.00,0.5,0.9,H{delta},0.1060,0.00,J0306395+000343,03 06 39.51,+00 03 44.7
71,72,3791.53,582,8.5,200.9,4.2,16.09,i,1.00,0.7,1.4,[NeV]3426,0.1067,0.00,J0306395+000343,03 06 39.53,+00 03 45.3
72,73,4126.75,356,36.4,437.0,4.8,16.09,i,1.00,1.7,3.3,[OII],0.1073,0.00,J0306395+000343,03 06 39.53,+00 03 45.0
73,74,4282.36,885,26.4,449.6,4.5,16.09,i,1.00,1.9,3.5,[NeIII]3869,0.1068,0.00,J0306395+000343,03 06 39.54,+00 03 44.6
74,75,5490.35,504,66.0,583.2,3.9,16.09,i,1.00,4.0,6.2,[OIII]4959,0.1071,0.00,J0306395+000343,03 06 39.54,+00 03 44.2
75,76,5381.88,1187,83.5,1264.5,3.4,16.09,i,1.00,8.3,13.3,H{beta},0.1072,0.00,J0306395+000343,03 06 39.54,+00 03 44.2


In [ ]:
#changing/appending data 
#saw already how to rename columns but here is how to add/remove columns 

## Building groups
* you can build groups based on column values 
* This could be really useful in this example to group by the line type (Trans column)
* pandas provides many built-in operations on groups--e.g., mean, count, etc.
* see http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-object-attributes

In [ ]:

# Split the dataset on Trans. This creates a pandas GroupBy object.
lines = hps_pd.groupby('Trans')

# You can call one of the groups by the name
NeV_lines = lines.get_group("[NeV]3426")
print(NeV_lines)

lines['Flux'].mean()

# The above line can also be more explicitly written as:
# groups['age'].aggregate('mean')

## Types of joins
* Joins allow you to join together different dataframes based on a key values (similar to SQL)
* for more information see the [merging documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html)
* `left`  joins - Use keys from left frame only
* `right` joins - Use keys from right frame only
* `outer` joins - Use union of keys from both frames
* `inner` joins - Use intersection of keys from both frames

In [ ]:
#Load in new a table from a paper to merge with the HPS catalog
#Bridge et. al. 2014 derived properties from SED fitting for the OII emitters in the HPS catalog

bridge_cols=['HPS_name', 'logM', 'logSFR', 'E(B-V)', 'OII_z']
bridge = pd.read_table('bridge2015_cat.ascii', delim_whitespace=True, names=bridge_cols, skiprows=16)
bridge

In [ ]:
#We want to join this table with the HPS catalog
#They share the common key (HPS id number) but they have different names in each table

#We first change the name of the HPS id in the bridge dataframe
bridge.rename(columns={'HPS_name':'HPS'}, inplace=True)

#We can then merge the two dataframes on the key HPS
#We use and inner join so that we only get rows back that have both HPS and bridge data
#If we use any other join it would fill in the missing data with NaNs 
HPS_OII = pd.merge(hps_pd, bridge, how='inner', on=['HPS'])
HPS_OII

#### Here is an example of a outer join
* In this case we want to preserve all of the rows in the HPS catalog 
* but when we can we want to add a value for the field that emission line was found in

In [ ]:
#read in the table from the vizier catalog that contains the information on the fields observed 
hps_fields = pd.read_table('hps_fields.txt', delimiter=None, header=50, skiprows=[52,53])
print(hps_fields)

#this function just finds the hour of the RA in the column 'RAJ2000'
def find_RA_hour(row):
    ra_h = row['RAJ2000'].split()[0] 
    return ra_h

#apply this function to build an RA_hour column in the hps dataframe
hps_pd['RA_hour'] = hps_pd.apply(find_RA_hour, axis=1)
print(hps_pd['RA_hour'].unique())

#apply this function to build an RA_hour column in the hps_fields dataframe
hps_fields['RA_hour'] = hps_fields.apply(find_RA_hour, axis=1)

#we only want to add the field name to the hps dataframe so this is just the Field column and the RA_hour to merge on
fields = hps_fields.loc[:,['Field','RA_hour']]

#We can then join the two databases to add the field column to the hps dataframe
hps_join_fields = pd.merge(hps_pd, fields, how='outer', on=['RA_hour'])
hps_join_fields.head(10)

### Creating crosstabs 
* This is a useful tool that lets you analzye columns against eachother 

In [ ]:
#creating crosstabs from groups
ctab = pd.crosstab(hps_join_fields['Trans'], hps_join_fields['Field'])

#You can also normalize the crosstab
#ctab = pd.crosstab(hps_join_fields['Trans'], hps_join_fields['Field'], normalize='index')

order = ctab.sum(1).sort_values(ascending=False).index
ctab = ctab.loc[order, :]

ctab

## Saving dataframes 
* This can be done in one line with both pandas and astropy tables 
* For the most part the format of the output does not matter since it is just going to be loaded in as a dataframe later
* in Pandas you can save the dataframe as a .csv or pickle file (may be more efficient for large files) 
* There are a lot more options for astropy tables 
    * basically any file you can load in you can save that file as the any of those outputs
    * the most useful thing is you can save a datafame as a latex deluxe table 

In [ ]:
#saving dataframes in Pandas 
HPS_OII.to_csv('bridge_HPS_cat.csv')

#saving astropy table as a deluxe latex table 
#Here is an example of how we could make a table of just the possible AGN
hps_at_OII = hps_at[hps_at['Trans'] == '[OII]']
hps_at_OII.write('HPS_OII_table.tex', format='ascii.aastex')

## Pandas makes it easy to do statistics on columns and the entire dataframe
* The describe function we saw earlier is customizable 

In [ ]:
#statistics with panadas 

#print(hps_pd.query('Trans == "[OII]"')['EW'].mean())
#print(hps_pd.query('Trans == "[OII]"')['EW'].std())

# Describe the continuous variables in the dataset.
# We can also include categoricals, or select other percentiles.
#hps_pd.describe(percentiles=[0.1, 0.9]).round(2)

lines.get_group('[OII]').describe()

## Data visualization using Pandas dataframes

#### Easy plotting with Matplotlib using Pandas columns

In [ ]:
#It is easy to plot in matplotlib by giving it pandas columns 

# Scatterplot of mass vs. SFR for the OII emitters from the Bridge et. al. 2014 paper
plt.scatter(HPS_OII['logM'], HPS_OII['logSFR'], color='green', s=30)
plt.xlabel("log(M)")
plt.ylabel("log(SFR)")
plt.title("Mass vs. SFR for the OII emitters in HPS", fontsize=16);

#### Plotting in Seaborn with dataframes

In [ ]:
#The crosstab we made can be easily plotted as a headmap 
ax = sns.heatmap(ctab[:12], annot=True, fmt='d', linewidths=.5,
                 annot_kws={'size': 14}, cmap='Blues')

# Customize the label size and figure size in base matplotlib.
# Notice we can access all axis and figure properties through
# # the handle we saved when calling seaborn.
ax.tick_params(labelsize=14)
ax.figure.set_size_inches((7, 8))  # update the plot size

In [ ]:
# Plot a heatmap of pairwise correlations between all continuous variables
r = HPS_OII.corr().round(2)

with sns.plotting_context("notebook", font_scale=1.1):
    ax = sns.heatmap(r, annot=True)
    ax.figure.set_size_inches((12, 9.5))

In [ ]:
#pairplots from a select group of columns for the OII catalog

tmp_data = HPS_OII.loc[:,['Flux', 'logM', 'logSFR', 'E(B-V)','z']]

with sns.plotting_context("notebook", font_scale=1.3):  # Make text bigger
    sns.pairplot(tmp_data, diag_kind='hist', kind='reg')

#### Plotting histograms in Pandas 

In [ ]:
#Only want to plot histogram for lines with more than 15 detections
many_detections = hps_pd.groupby('Trans').filter(lambda x: len(x)>15)

# Plot flux histograms for all line types--with shared x-axis
many_detections['Flux'].hist(by=many_detections['Trans'], bins=20, sharex=True);

# Embiggen the figure a bit
plt.gcf().set_size_inches((10, 10))